In [1]:
%load_ext autoreload

In [2]:
import os
import typing

import joblib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import tensorflow as tf
from IPython.display import display
from PIL import Image, ImageFilter
from tensorflow.keras.layers import Layer
from tensorflow.keras.losses import Loss
from tensorflow.keras.models import Model, Sequential

from util.models import new_model
from util.func.sobel_funcs import SobelEdges

from PIL import Image, ImageFilter

2022-12-12 17:54:48.284398: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 17:54:48.349887: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
%autoreload 2
%aimport util.models
%aimport util.func.sobel_funcs

In [4]:
class SobelLayer(tf.keras.layers.Layer):
    def __init__(self, blur, activation=None):
        super().__init__()
        self.activation = activation
        self.edgefinder = sobel_edge_finder
        self.blur = blur

    # def build(self, input_shape):
    #     self.w = self.add_weight(
    #             shape=(input_shape[-1]),
    #             initializer="random_normal",
    #             trainable=True)
    #             
    #     self.b = self.add_weight(
    #             shape=(1,), initializer="zeros", trainable=True)

    def call(self, input):
        blur = self.blur
        print(input.shape)
        x = self.edgefinder(input)
        x = x.hsv_edges()
        return tf.add(input, x)

In [5]:
class sobel_edge_finder():
    def __init__(self, input:np.ndarray):
        self.input = input
        self.width, self.height, _ = input.shape
        self.channels = 1
        # self.blur_amount = blur
        self.x_g, self.y_g = self._process_edges()
        self.angle_map = np.arctan2(self.y_g, self.x_g)

    def _process_edges(self):
        # image = Image.fromarray(self.input)
        # blurred = image.filter(ImageFilter.GaussianBlur(self.blur_amount))
        # blurred_arr = np.array(blurred, dtype='float')
        # blurred_tensor = tf.reshape(tf.convert_to_tensor(blurred_arr, dtype=tf.float32), [1,
        #                                                                  self.width,
        #                                                                  self.height,
        #                                                                  self.channels])
        edge_tensor = tf.image.sobel_edges(self.input)
        x_g = np.asarray(edge_tensor[0, :, :, :, 0])
        y_g = np.asarray(edge_tensor[0, :, :, :, 1])

        return (x_g, y_g)
    
    def hsv_edges(self):
        """
        This method combines the angle_map and bw_edges attributes into a hsv map where the sin of 
        the gradient is used as the hue and the magnitude of the gradient is the value attributes 
        of an hsv image tensor. That tensor is then converted into an rgb image for display.

        returns tf.tensor with shape (width, height, 3)
        """
        hue = (self.angle_map + np.pi) / (np.pi * 2)                            # angle_map is in radians from - to  this remaps it to 0 to 1
        saturation = np.ones((self.width, self.height, 1), dtype='float')
        value = self.bw_edges()
        value = (value - value.min()) / value.max()

        return np.concatenate((hue, saturation, value), axis=2) 
        
    def bw_edges(self):
        """
        Returns a grayscale representation of the sobel edges.
        """
        return np.sqrt(self.x_g**2 + self.y_g**2)

In [6]:
image = "./data/screw/train/good/002.png"
input = Image.open(image)
input = np.asarray(input.resize((256, 256)))
print(type(input))
edges = SobelEdges(image, blur=4, resize=(256, 256))
hsv_edges = edges.hsv_edges()
print(type(hsv_edges), hsv_edges.shape)

<class 'numpy.ndarray'>


2022-12-12 17:54:55.703251: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1026] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-12 17:54:55.705631: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1026] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-12 17:54:55.705655: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1026] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-12 17:54:55.707871: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1026] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-12 17:54:55.707897: I tensorflow/com

<class 'numpy.ndarray'> (256, 256, 3)


In [10]:
img_size = (256, 256)
mut_img_size = img_size + (3,)
print(img_size)

(256, 256)


In [11]:
input = (input - input.min()) / input.max()
input = np.reshape(input, (*img_size, 1))
layer_input = np.concatenate((input, hsv_edges), axis=2)
print(input.shape, layer_input.shape)

(256, 256, 1) (256, 256, 4)


In [8]:
def conv_layers(img_size):
    inputs = tf.keras.Input(shape=(img_size + (4,)))
    x = tf.keras.layers.Conv2D(8, 3, activation="relu", padding="same", strides=2)(inputs)
    x = tf.keras.layers.Conv2D(8, 3, padding="same", strides=2)(x)
    x = tf.keras.layers.Conv2D(48, 3, activation="relu", padding="same", strides=2)(x)
    x = tf.keras.layers.Conv2D(48, 3, padding="same", strides=2)(x)
    residual = x
    x = tf.keras.layers.Conv2D(128, 3, activation="relu", padding="same", strides=2)(x)
    x = tf.keras.layers.Conv2D(128, 3, padding="same", strides=2)(x)

    x = tf.keras.layers.Conv2DTranspose(96, 3, padding="same", strides=2)(x)
    x = tf.keras.layers.Conv2DTranspose(48, 3, activation="relu", padding="same", strides=2)(x)
    x = tf.add(x, residual)
    x = tf.keras.layers.Conv2DTranspose(24, 3, padding="same", strides=4)(x)
    x = tf.keras.layers.Conv2DTranspose(16, 3, activation="relu", padding="same", strides=4)(x)

    outputs = tf.keras.layers.Conv2D(4, 3, activation="softmax", padding="same")(x)

    model = tf.keras.Model(inputs, outputs)
    return model

model = conv_layers(img_size=img_size)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 4  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 8)  296         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 64, 64, 8)    584         ['conv2d[0][0]']                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 32, 32, 48)   3504        ['conv2d_1[0][0]']           

In [12]:
inputs = np.reshape(layer_input, (1, *img_size, 4))
model.compile(optimizer='rmsprop', loss='mse')

In [13]:
# os.environ["XLA_FLAGS"]="--xla_gpu_cuda_data_dir=/opt/cuda"
os.environ["XLA_FLAGS"]="--xla_gpu_cuda_data_dir=/home/drew/conda/envs/tf_env/"
history = model.fit(inputs, inputs,
                    epochs=1,
                    validation_data=(inputs, inputs))

2022-12-12 17:55:18.960609: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops_fused_impl.h:675 : UNKNOWN: CUDNN_STATUS_EXECUTION_FAILED
in tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc(4644): 'status'


UnknownError: Graph execution error:

Detected at node 'model/conv2d/Relu' defined at (most recent call last):
    File "/home/drew/conda/envs/tf_env/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/drew/conda/envs/tf_env/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/drew/conda/envs/tf_env/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/drew/conda/envs/tf_env/lib/python3.10/site-packages/traitlets/config/application.py", line 985, in launch_instance
      app.start()
    File "/home/drew/conda/envs/tf_env/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/drew/conda/envs/tf_env/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/drew/conda/envs/tf_en

In [ ]:
print(x[0,:,:,1].shape)
fig = plt.Figure()
ax = plt.subplot()
ax.imshow(mcolors.hsv_to_rgb(hsv_edges), cmap='gray')
plt.show()